# Урок 4. Оценка и интерпретация полученной модели. Обсуждение курсового проекта

## 1. Расскажите, как работает регуляризация в решающих деревьях, какие параметры мы штрафуем в данных алгоритмах?

Регуляризация:
- минимизирует функционал ошибки с ограничением весов;
- "штрафует" модель за слишком большие веса путем добавления нового члена к ошибке:
$$Q(w, X) + \lambda ||w||^{2} \rightarrow \underset{w}{\text{min}};$$
- позволяет контролировать рост модели опираясь на анализ ее сложности;
- помогает решить проблему переобученности модели;
- работает либо по принципу раннего останова либо по принципу упрощения дерева (pruning).

Параметр $\lambda$ позволяет управлять степенью строгости штрафа (подбираем по кросс-валидации):

- чем меньше значение $\lambda$, тем больше штраф за признак с высоким весом. В процессе такого обучения получается **большая** сложность модели. Если уменьшать значение $\lambda$, в какой-то момент появляется вероятность чрезмерного усложнения модели и переобучения;

- чем больше значение $\lambda$, тем меньше штраф за признак с высоким весом. В процессе такого обучения получается **меньшая** сложность модели. Если увеличивать значение $\lambda$, в какой-то момент оптимальным для модели окажется зануление всех весов.


При подходе раннего останова штрафуются, например:

- max_depth - максимальная глубина дерева; 

- min_samples_leaf - минимальное число объектов в листе. Например, если он равен 5, то дерево будет порождать только те классифицирующие правла, которые верны как мининмум для 5 объектов. Можно задать числом или процентом от общего числа объектов (по дефолту — 1);

- min_samples_split - минимальное количество объектов, необходимое для разделения внутреннего узла. Можно задать числом или процентом от общего числа объектов (по дефолту — 2);

- max_features - максимальное чило признаков, по которым ищется лучшее разбиение в дереве (необходимо из-за того, что при большом количестве признаков будет "дорого" искать лучшее (по критерию типа прироста информации) разбиение среди всех признаков).

При подходе упрощения дерева (pruning: дерево сначала полностью строится, затем уменьшается до оптимального с точки зрения достижения максимальной обучающей способности размера путем объединения некоторых концевых вершин) штрафуются, например, при применении:

а) XGBoost: importance_type
$$Gain - \gamma \geqslant 0 \Rightarrow оставляем$$
$$Gain - \gamma < 0 \Rightarrow убираем;$$

б) LightGBM: max_depth;

в) CatBoost: max_ctr_complexity, store_all_simple_ctr;

г) Cost Complexity Pruning- Alpha: ccp_alpha.

## 2. По какому принципу рассчитывается "важность признака (feature_importance)" в ансамблях деревьев?

На каждом шаге построения дерева выбирается признак и его значение, которые дают наилучшее разбиение данных. В зависимости от алгоритма, применяются разные способы подсчета прироста информации, чтобы вычислить наилучшее разбиение(например считают критерий Джини или энтропию в каждой части разбиения). Важность признака в дереве определяется степенью снижения меры неопределенности, т.е. как раз критерия Джини или энтропии и других. Чем выше признак в дереве, тем он важнее. В ансамблях деревьев важность признака считается как среднее арифметическое по всем важностям данного признака для каждого дерева. В случае, если признак не присутствует в дереве, его важность считается нулевой.

Методами вычисления важности признаков в алгоритмах ансамблей деревьев являются:

- MDI (Mean Decrease in Impurity, Gini Importance). Чем больше, тем выше важность. Данный метод уменьшает количество значений признака и вычисляет точность модели после изменений. Чем сильнее уменьшение точности модели, тем выше важность признака. Может некорректно отображать важность, если признак содержит большое количество уникальных значений;

- MDA (Mean Decrease in Accuracy, Permutation Importance). Данный метод перемешивает значения в признаке и измеряет точность модели. В отличие от MDI не имеет проблем с признаками с большим количеством уникальных значений;

- Boruta. Данный метод так же как MDA перемешивает значения в признаках, но перемешанные данные добавляются в основной датасет.